In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np
import os

os.environ["PYSPARK_PYTHON"] = '/usr/bin/python3.6'
os.environ["PYSPARK_DRIVER_PYTHON"] = '/usr/bin/python3.6'

from pyspark import SparkConf

conf = SparkConf().setAppName('cluster') \
    .setMaster('spark://10.10.1.1:7077') \
    .set('spark.task.cpus', '16') \
    .set('spark.executor.memory', '124g')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.addPyFile("cerebro.zip")

work_dir = '/var/nfs/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=6)
store = LocalStore(prefix_path=work_dir + 'test/')

CEREBRO => Time: 2021-12-06 22:40:53, Running 6 Workers


In [2]:
feature_columns=['features']
label_columns=['label']

df = spark.read.load(work_dir + "IMDB/data.parquet")
df.show(5)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=100)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|I've always been ...|    1|
|This movie was Hi...|    1|
|All you could eve...|    1|
|This movie is hil...|    1|
|BIG FAT LIAR, in ...|    1|
+--------------------+-----+
only showing top 5 rows



In [3]:
from keras_tuner.engine import hyperparameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel

img_shape = (28, 28, 1)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="greedy", 
#     tuner="randomsearch",
    hyperparameters=None, 
    objective="val_accuracy",
    max_trials=2,
    overwrite=True,
    exploration=0.3,
)

In [4]:
ms = am.tuner.model_selection
backend = ms.backend
_, _, metadata, _ = backend.prepare_data(ms.store, train_df, ms.validation, label_columns=ms.label_cols, feature_columns=ms.feature_cols)

CEREBRO => Time: 2021-12-06 21:27:38, Num Partitions: 192
CEREBRO => Time: 2021-12-06 21:27:38, Writing DataFrames
CEREBRO => Time: 2021-12-06 21:27:38, Train Data Path: file:///var/nfs/test/intermediate_train_data
CEREBRO => Time: 2021-12-06 21:27:38, Val Data Path: file:///var/nfs/test/intermediate_val_data
CEREBRO => Time: 2021-12-06 21:27:42, Train Partitions: 153


Py4JJavaError: An error occurred while calling o122.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [4]:
train_df.dtypes

[('features', 'string'), ('label', 'bigint')]

In [11]:
from cerebro.backend.spark.util import *

unischema_fields = []
metadata = _get_metadata(train_df)
for k in metadata.keys():
    type = spark_to_petastorm_type(metadata[k]['spark_data_type'])
    shape = petastorm_unischema_shape(metadata[k]['shape'])
    codec = petastorm_unischema_codec(metadata[k]['shape'], metadata[k]['spark_data_type'])
    unischema_fields.append(UnischemaField(k, type, shape, codec, False))

SyntaxError: invalid syntax (<ipython-input-11-2afdb75b95e2>, line 1)

In [ ]:
metadata

In [12]:
unischema_fields

[UnischemaField(name='features', numpy_dtype=<class 'numpy.uint8'>, shape=(), codec=<petastorm.codecs.NdarrayCodec object at 0x7f84008c9908>, nullable=False),
 UnischemaField(name='label', numpy_dtype=<class 'numpy.int64'>, shape=(), codec=<petastorm.codecs.NdarrayCodec object at 0x7f8259d71550>, nullable=False)]

In [13]:
from petastorm.unischema import Unischema, UnischemaField, dict_to_spark_row
petastorm_schema = Unischema('petastorm_schema', unischema_fields)

In [14]:
print(petastorm_schema)

Unischema(petastorm_schema, [
  UnischemaField('features', uint8, (), <petastorm.codecs.NdarrayCodec object at 0x7f84008c9908>, False),
  UnischemaField('label', int64, (), <petastorm.codecs.NdarrayCodec object at 0x7f8259d71550>, False),
])


In [15]:
print(train_df.rdd \
      .map(lambda x: x.asDict()) \
      .take(1))

[{'features': '"Big Fat Liar" comes as a welcome -- shallow, but welcome -- breath of fresh air after one too many films featuring bathrooms, bodily fluids, pets on acid, gaseous jokes and crotch gags. After all, "See Spot Run," "Max Keeble\'s Big Move" and "Snow Dogs" had signaled The Degenerative Spiral of Kiddie Movie-making. Worse is the realization that the young audience would later be satiated by the smuttier offerings of "Not Another Teen Movie" and "Slackers." Written by Nickelodeon producers Dan Schneider and Brian Robbins (who coincidentally co-starred on the \'80s sitcom "Head of the Class"), the film stars Frankie Muniz as Jason, who\'s always ready with a good lie to get out of scrapes. (Where\'s the English paper? Dad choked on a Swedish meatball.) His teacher (Sandra Oh) issues an ultimatum: Turn in the paper by the end of the day or take summer school. Jason whips out a story called "Big Fat Liar" and is struck en route to school by a limousine carrying insensitive Hol

In [16]:
print(train_df.rdd \
      .map(lambda x: x.asDict()) \
      .map(lambda x: {k: create_array(x,k,spark_to_petastorm_type(metadata[k]['spark_data_type'])) for k in x}) \
      .take(1))

[{'features': array([ 34,  66, 105, ..., 111, 112,  46], dtype=uint8), 'label': array(1)}]


In [17]:
print(train_df.rdd \
      .map(lambda x: x.asDict()) \
      .map(lambda x: {k: create_array(x,k,spark_to_petastorm_type(metadata[k]['spark_data_type'])) for k in x}) \
      .map(lambda x: dict_to_spark_row(petastorm_schema, x))
      .take(1))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 4 times, most recent failure: Lost task 0.3 in stage 14.0 (TID 1927, 10.10.1.2, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.6/dist-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)

In [18]:
sf = petastorm_schema.fields['features']
sf

UnischemaField(name='features', numpy_dtype=<class 'numpy.uint8'>, shape=(), codec=<petastorm.codecs.NdarrayCodec object at 0x7f84008c9908>, nullable=False)

In [19]:
np.fromstring("abc", dtype=np.uint8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


array([97, 98, 99], dtype=uint8)

In [16]:
def create_array_1(x, k, dtype):
    if spark_to_petastorm_type(metadata[k]['spark_data_type']) == np.uint8:
        return np.frombuffer(x[k], dtype)
    else:
        return np.array(x[k], dtype=dtype)

In [15]:
np.array(1, np.int32)

array(1, dtype=int32)

In [33]:
np.array('sba')

array('sba', dtype='<U3')